In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import torchtext
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
print(torch.__version__)

C:\ProgramData\Anaconda3\lib\site-packages\mpmath\ctx_mp_python.py:892: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:
C:\ProgramData\Anaconda3\lib\site-packages\mpmath\ctx_mp_python.py:986: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:


2.0.0+cpu


In [7]:
class Embeddings(nn.Module):
    def __init__(self,d_model,vocab):
        super(Embeddings,self).__init__()
        self.lut = nn.Embedding(vocab,d_model)
        self.d_model = d_model
    def forward(self,x):
        return self.lut(x)*math.sqrt(d_model)

In [8]:
d_model = 512
vocab =1000 # words number of dictionary

In [9]:
em = Embeddings(d_model=d_model,vocab=vocab)
word = Variable(torch.LongTensor([[2,4,5,6],[100,200,300,400]]))

In [10]:
wordem =em(word)

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self,d_model,dropout,max_len):
        super(PositionalEncoding,self).__init__()
        self.dropout = nn.Dropout(p = dropout)
        pe = torch.zeros(max_len,d_model)
        position = torch.arange(0,max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0,d_model,2)*-math.log(10000)/d_model)
        pe[:,0::2] = torch.sin(position*div_term)
        pe[:,1::2] = torch.cos(position*div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe',pe)
    def forward(self,x):
        x = x+ Variable(self.pe[:,:x.size(1),:],requires_grad = False)
        return self.dropout(x)

In [12]:
d_model = 512
dropout = 0.1
max_len = 50

In [50]:
pe_vector = PositionalEncoding(d_model,dropout,max_len)

In [51]:
pe_result = pe_vector(wordem)

In [16]:
def subquent_mask(size):
    attn_shape = (1,size,size)
    subquent_mask = np.triu(np.ones(attn_shape),k=1).astype('uint8')
    return torch.from_numpy(1-subquent_mask)

In [17]:
size =5
mask_ma=subquent_mask(size)

In [18]:
def attention(query,key,value,mask=None,dropout =None):
    d_model = query.size(-1)
    scores = torch.matmul(query,key.transpose(-2,-1))/math.sqrt(d_model)
    if mask is not None:
        scores = scores.masked_fill(mask==0,-1e9)
    p_attn = F.softmax(scores,dim=-1)
    if dropout is not None:# dropout is none should be input with function nn.Module(p=0.1)
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn,value),p_attn

In [19]:
query = key = value = word_pe

mask = Variable(torch.zeros(2,4,4))

In [20]:
attn,p_attn = attention(query,key,value)

In [21]:
print(f"attn shape is {attn.shape}")

attn shape is torch.Size([2, 4, 512])


In [22]:
print(f"p_attn shape is {p_attn.shape}")

p_attn shape is torch.Size([2, 4, 4])


In [36]:
def clones(module,N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [40]:
class MultiHeadedAttention(nn.Module):
    def __init__(self,head,embedding_dim,dropout=0.1):
        super(MultiHeadedAttention,self).__init__()
        assert embedding_dim%head ==0
        self.d_k = embedding_dim//head
        self.head = head
        self.embedding_dim = embedding_dim
        self.linears = clones(nn.Linear(embedding_dim,embedding_dim),4)
        self.attn =None
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, query, key, value,mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        batch_size = query.size(0)
        query, key, value = [model(x).view(batch_size,-1,self.head,self.d_k).transpose(1,2)
                             for model,x in zip(self.linears, (query, key, value))]
        x,self.attn = attention(query, key,  value, mask =mask, dropout = self.dropout)
        x= x.transpose(1,2).contiguous().view(batch_size,-1,self.head*self.d_k)
        return self.linears[-1](x)

In [44]:
head = 8
embedding_dim = 512
dropout =0.2

In [52]:
query=key=value= pe_result

In [53]:
mask = Variable(torch.zeros(8,4,4))

In [54]:
mha = MultiHeadedAttention(head,embedding_dim,dropout)

In [55]:
pe_result.shape

torch.Size([2, 4, 512])

In [56]:
mha_result = mha(query,key,value,mask)

RuntimeError: The size of tensor a (8) must match the size of tensor b (2) at non-singleton dimension 0